# 💻 Geokoding i geopandas


Geopandas støtter geokoding via et bibliotek kalt
[geopy](http://geopy.readthedocs.io/), som må være installert for å bruke
[geopandas ' `geopandas.tools.geocode()`
funksjon](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html).
`geocode()` forventer en `liste` eller `pandas.Series` av adresser (strenger) og
returnerer en `GeoDataFrame` med løste adresser og punktgeometrier.

La oss prøve dette ut.

Vi vil geokode adresser lagret i en semikolon-separert tekstfil kalt
`adresser.txt`. Disse adressene ligger i alle i Oslo.

In [1]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

In [ ]:
import pandas




Vi har en `id` for hver rad og en adresse i `adr` kolonnen.


## Geokode adresser ved hjelp av *Nominatim*

I vårt eksempel vil vi bruke *Nominatim* som en *geokodingstilbyder*. [*Nominatim*](https://nominatim.org/) er et bibliotek og en tjeneste som bruker OpenStreetMap-data, og drives av OpenStreetMap Foundation. Geopandas '
[`geocode()`
funksjon](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html) støtter den innebygd.

```{admonition} God bruk
:class: note

[Nominatims brukervilkår](https://operations.osmfoundation.org/policies/nominatim/)
krever at brukere av tjenesten sørger for at de ikke sender mer hyppige
forespørsler enn en per sekund, og at en tilpasset **bruker-agent** streng er
knyttet til hver forespørsel.

Geopandas' implementering lar oss spesifisere en `user_agent`; biblioteket tar også
hånd om å respektere hastighetsbegrensningen til Nominatim.

Å slå opp en adresse er en ganske kostbar databaseoperasjon. Derfor er det,
noen ganger, den offentlige og gratise Nominatim-serveren bruker litt lenger tid å på
svare. I dette eksempelet legger vi til en parameter `timeout=10` for å vente opptil 10
sekunder for et svar.
```

In [ ]:
import geopandas


Et voilà! Som et resultat fikk vi tilbake en `GeoDataFrame` som inneholder en analysert
versjon av våre originale adresser og en `geometry` kolonne med
`shapely.geometry.Point`s som vi kan bruke, for eksempel, til å eksportere dataene til
et romlig dataformat.

Imidlertid ble `id`-kolonnen forkastet i prosessen. For å kombinere inputdatasettet med resultatsettet vårt, kan vi bruke pandas' [*join*
operasjoner](https://pandas.pydata.org/docs/user_guide/merging.html).


## Koble sammen dataframes

:::{admonition} Koble sammen datasett ved hjelp av pandas
:class: note

For en omfattende oversikt over forskjellige måter å kombinere dataframes og
Series, ta en titt på pandas dokumentasjon om [merge,
join og
concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).
:::


Å koble data fra to eller flere dataframes eller tabeller er en vanlig oppgave i mange
(romlige) dataanalysearbeidsflyter. Kombinering av data fra forskjellige tabeller basers på en felles **nøkkel**-attributt gjøres enkelt i pandas/geopandas ved hjelp av [`merge()`
funksjonen](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

Men, noen ganger er det nyttig å koble to dataframes sammen basert på deres
**indeks**. Dataframes må ha **samme antall rader** og
**dele den samme indeksen** (enkelt forklart, de skal ha samme rekkefølge av rader).

Vi kan bruke denne tilnærmingen, for å koble informasjon fra den originale dataframen `adresser` til de geokodede adressene `geokodede_adresser`, rad for rad.
`join()`-funksjonen, som standard, kobler to dataframes basert på indeksen deres.
Dette fungerer for eksemplet vårt, da rekkefølgen på de to dataframesene er identiske.

Utdataen fra `join()` er en ny `geopandas.GeoDataFrame`:

Den nye data rammen har alle originale kolonner pluss nye kolonner for `geometry`
og for en analysert `adresse` som kan brukes til å spot-sjekke resultatene.

:::{note}
Hvis du skulle gjøre join den andre veien, dvs. `adresser.join(geokodede_adresser)`, ville utdata være en `pandas.DataFrame`, ikke en `geopandas.GeoDataFrame`.
:::


---


Det er nå enkelt å lagre det nye datasettet som en geospatial fil, for eksempel, i
*GeoPackage* format:

In [6]:
# slett en muligens eksisterende fil, da den skaper
# problemer i tilfelle sphinx kjøres gjentatte ganger
try:
    (DATA_MAPPE / "oslo_adresser" / "adresser.gpkg").unlink()
except FileNotFoundError:
    pass